In [1]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
from typing import Dict, Text

In [2]:
ratings = tfds.load("movielens/100k-ratings", split = "train")
movies = tfds.load("movielens/100k-movies", split = "train")

2022-09-25 17:54:47.139526: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
ratings = ratings.map(lambda x : {
    "user_id" : x["user_id"],
    "movie_title" : x["movie_title"]})
movies = movies.map(lambda x : x["movie_title"])

In [4]:
tf.random.set_seed(42)

shuffled = ratings.shuffle(100000, seed = 42, reshuffle_each_iteration= False) # Forgot to specify the amount of times to shuffle

#Forgot to make train and test
train = shuffled.take(80000)
test = shuffled.skip(80000).take(20000)


movie_titles = movies.batch(1000)
user_ids = ratings.batch(1000000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [5]:
user_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(vocabulary=unique_user_ids, mask_token=None),
    tf.keras.layers.Embedding(len(unique_user_ids)+1, 32)
])

movie_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(vocabulary=unique_movie_titles, mask_token = None),
    tf.keras.layers.Embedding(len(unique_movie_titles)+1, 32)
])

In [6]:
task = tfrs.tasks.Retrieval(tfrs.metrics.FactorizedTopK(candidates= movies.batch(128).map(movie_model))) # Forgot candidates argument